In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
import pickle
import umap
import spacy
from tqdm import tqdm

In [2]:
# model_path='/data/model/'
model_path='model/'

In [3]:
import psutil
psutil.virtual_memory()

svmem(total=59098333184, available=55813578752, percent=5.6, used=2673225728, free=53153333248, active=2641756160, inactive=2720854016, buffers=58015744, cached=3213758464, shared=9109504, slab=204210176)

In [9]:
df_reader = pd.read_csv('parsed/corpus/all.csv', usecols=['text'], chunksize=10000)
docs_gen = (x for df in df_reader for x in df.text.to_list())

In [5]:
with open('polish_stopwords.txt') as f:
    stop_words = [x.strip() for x in f]

nlp = spacy.load('pl_spacy_model')

def tokenize(text):
    analysis = nlp(text)
    return [token.lemma_ for token in analysis]

tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 3), min_df=5, max_df=0.1)

In [12]:
embeddings_tfidf = tfidf_vectorizer.fit_transform(tqdm(docs_gen, total=330490))

  0%|          | 602/330490 [05:10<48:53:53,  1.87it/s]

In [6]:
with open(f'{model_path}embeddings-tfidf-2015-morf.pkl', "wb") as fOut:
    pickle.dump(embeddings_tfidf, fOut)

In [40]:
embeddings_tfidf.shape

(41930, 290255)

In [12]:
feature_names = np.array(tfidf_vectorizer.get_feature_names())

def get_top_tf_idf_words(response, top_n=5):
    sorted_nzs = np.argsort(response.data)[:-(top_n+1):-1]
    return feature_names[response.indices[sorted_nzs]]

tfidf_words = [get_top_tf_idf_words(x) for x in embeddings_tfidf]

In [13]:
df['tfidf_words'] = tfidf_words

In [21]:
with open(f'{model_path}tfidf-words-max03.pkl', "rb") as fIn:
    tfidf_words_cmp = pickle.load(fIn)

df_all['tfidf_words_cmp'] = tfidf_words_cmp

In [22]:
df['tfidf_words_cmp'] = df_all[df_all.rok=='2015'].tfidf_words_cmp

In [39]:
df[['text','tfidf_words','tfidf_words_cmp']].iloc[3293].to_list()

['Panie Marszałku! Panie Ministrze! Wysoka Izbo! Procedując dzisiaj projekt nowelizacji ustawy, przekonujemy się po raz kolejny, jak trudnym elementem funkcjonowania, realizacji zadań przez nasz system opieki zdrowotnej jest moment kontraktowania procedur. Ze swego doświadczenia mam w pamięci już wielokrotne finał roku, zakończenie roku, czas na kontraktowanie, przedłużanie bądź uczestniczenie podmiotów w procedurze konkursowej. Wiązało się to niejednokrotnie z wielkimi emocjami. Analizując aktualną sytuację i zapowiedzi rządu co do zmiany podmiotu, co do sposobu zarządzania, co do zmiany płatnika, zmian organizacyjnych w resorcie, uważamy, że proponowane zmiany mają szeroki związek właśnie z tą przygotowywaną reformą. Nie ukrywam, że jeślibyśmy poszli tym torem, to obdarzając zaufaniem, przyjmując określone propozycje, możemy domniemywać, że jest to również czas czy okres, czasokres, który zostanie poświęcony na głęboką analizę mechanizmów, które funkcjonują i pozwalają zapewnić reali

In [9]:
with open(f'{model_path}tfidf-words-max300k.pkl', "wb") as fOut:
    pickle.dump(tfidf_words, fOut)

In [10]:
embeddings_tfidf.shape

(330490, 3300133)

In [12]:
word_weights = dict(zip(tfidf_vectorizer.get_feature_names(), tfidf_vectorizer.idf_))

In [22]:
with open(f'{model_path}tfidf-word-weights-max03.pkl', "wb") as fOut:
    pickle.dump(word_weights, fOut)

# Reduce dims

In [4]:
with open(f'{model_path}embeddings-tfidf-all-max300k.pkl', "rb") as fIn:
    embeddings_tfidf = pickle.load(fIn)

In [41]:
embeddings_reduced = umap.UMAP(n_neighbors=15,
                    n_components=5,
                    min_dist=0.0,
                    metric='hellinger').fit_transform(embeddings_tfidf)

In [42]:
with open(f'{model_path}embeddings-tfidf-2015-morf-5d.pkl', "wb") as fOut:
    pickle.dump(embeddings_reduced, fOut)